## Computer vision workshop

Welcome to this workshop on Computer Vision and Convolutional Neural Networks. In a couple of steps we explain how computer vision techniques can be used to manipulate and analyse images and how convolutional neural networks use transformations to analyse visual material.

## Instructions
Prior knowledge of Python (a computer language) is not needed, but will help you understand this notebook better. The same goes for Jupyter Notebooks. One important thing to understand: a Jupyter notebook consists of cells that can  contain blocks of code or text ('markdown'). For example, this block contains markdown only. In cells that contain code, we can add text as comments using the '#' sign. This text is used to explain code. Sometimes we ask a question after an '#' sign. If you want to write your answer down in the same cell, use an # to signify it as text. 

In order to make the code work you need to run code in the cells in a top to bottom order. You do this by pressing 'shift + enter', which will automatically take you to the next cell. Try it now!

NB: This notebook builds upon earlier notebooks/work by <br>
<a href="https://github.com/SeguinBe/DHWorkshop2017" /> Benoit Séguin </a> <br>
<a href="https://distantviewing.org/tutorial-dl" /> Taylor Arnold & Lauren Tilton </a>


In [ ]:
# We start by loading several libraries we need to perform the tasks in this notebook. 
# Dont forget to press 'Shift + enter'

import pandas as pd 
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

import os
from os.path import join

import skimage
from skimage import data, io

In [ ]:
# Preloading models
from keras.datasets import mnist

from keras.applications.vgg19 import VGG19
vgg19_full = VGG19(weights='imagenet')

from keras.applications import inception_v3
inception_model = inception_v3.InceptionV3(include_top=False, pooling='avg', weights='imagenet')

## Images and Python

We start by loading an image in the notebook using Python: in this case, the image with the filename 'color_train.jpg' from the folder 'images' (see the code below). `plt.imshow(img)` shows the image in the notebook in a grid, which shows the number of pixels. 

In [ ]:
#Load an image using skimage. 
img = io.imread(join("images", "color_train.jpg"))
plt.imshow(img)

## Pixels
Computers do not see images, but only a large number of pixel values. Each pixel has a value between 0 and 255. As a result, computers are ('only') able to analyse patterns in pixels values. 
<br><br><br><img src="https://openframeworks.cc/ofBook/images/image_processing_computer_vision/images/lincoln_pixel_values.png" /><p style="text-align:center;">
Pixel data diagram. At left, our image of Lincoln; at center, the pixels labeled with numbers from 0-255, representing their brightness; and at right, these numbers by themselves.
</p>
<p style="text-align:center;">
(borrowed from Golan Levin [Image Processing and Computer Vision](https://openframeworks.cc/ofBook/chapters/image_processing_computer_vision.html))</p>

In [ ]:
# We can use 'img.shape' to print information on the number of pixels.
# QUESTION: What do the numbers stand for? Look at the image, and the grid pattern, in the image in cell 6. 
# QUESTION: What does the 3 mean? 
img.shape

## Color Layers

In [ ]:
# An image can consist of three color layers: red, green, and blue, all with their own pixel values.

img_rgb = io.imread(join("images", "iur.png"))
plt.imshow(img_rgb)

red = img_rgb[:,:,0] #select the first channel and save into variable
green = img_rgb[:,:,1]
blue = img_rgb[:,:,2]

In [ ]:
plt.figure(figsize=(20,20))
plt.subplot(1,3,1) #plot one figure with 1 row and 3 columns of images.
plt.title('Red')
plt.imshow(red, cmap='gray')
plt.axis('off') # plot no axis
plt.subplot(1,3,2)
plt.title('Green')
plt.imshow(green, cmap='gray')
plt.axis('off')
plt.subplot(1,3,3)
plt.title('Blue')
plt.imshow(blue, cmap='gray')
plt.axis('off')

## Transformations

Because computers see images as collections of numbers, we can use math to transform the pixel values and, as a result, manipulate the image. 

In [ ]:
# Imports the necessary library
from skimage import transform

In [ ]:
# First we will rescale the image by a factor 0.5 (divide the number of pixels on the X and Y axel by 2)
# QUESTION: The size of the displayed image did not change. Why? What did change? 
rescaled_img = transform.rescale(img, scale=0.5, mode='constant', multichannel=True, anti_aliasing=True)
plt.imshow(rescaled_img)
rescaled_img.shape

In [ ]:
# Now we will resize the image to have a size of 200 x 200 pixels. 
# Before applying Convolutional Neural Networks, images are often 'preprocessed' in this, and other, ways.
# QUESTION: resizing the image in this way fundamentally changes our perception of the image. 
# Is this the same for a computer?
resized_img = transform.resize(img, (200,200), mode='constant', anti_aliasing=True)
plt.imshow(resized_img)
resized_img.shape

In [ ]:
# We can also manipulate the pixels to rotate image by 90° counter-clockwise.
rotated_img = transform.rotate(img, 90, resize=True)
plt.imshow(rotated_img)

In [ ]:
# Question: Resize the image by a factor 3 and rotate the image by 45° counter-clockwise.
# Re-use the code from the cells above, or write your own.

## Convolutions

Convolutions are another way to manipulate the pixel values of images. You can imagine the mathemetical operation of a convolution (https://en.wikipedia.org/wiki/Convolution) as a small screen, known as a kernel, running over the image. The kernel changes the value of the pixel in the middle based on the values of the pixels surrounding it. As we will see below, convolutions highlight certain features of an image. Convolutional Neural Networks use convolutions to extract features --- complicated patterns in pixel values. Below we apply several convolutions to see which features they highlight. 

<br><br><br><img src="https://community.arm.com/cfs-file/__key/communityserver-blogs-components-weblogfiles/00-00-00-20-66/4786.conv.png" /><p style="text-align:center;">
An example of a convolution filter.  Each transformed pixel value is created by multiplying its current value and the values of the pixels around it against a matrix of coefficients
</p>
<p style="text-align:center;">
(borrowed from [Embarrassingly Parallel](https://community.arm.com/graphics/b/blog/posts/when-parallelism-gets-tricky-accelerating-floyd-steinberg-on-the-mali-gpu))</p>

In [ ]:
# This cell loads the necessary libraries
from scipy.ndimage.filters import convolve
from scipy import ndimage as ndi
from scipy.signal import convolve2d
from scipy import misc
from skimage import filters

### Blur (Guassian Filter)

In [ ]:
# Here we initialize a matrix of 100 x 100 pixels with a value zero. 
# And we also add a pixel of value 1 at position 50, 50.
x = np.zeros((100, 100))
x[50, 50] = 1
plt.imshow(x, interpolation='none', cmap='gray')

### Gaussian Filter

We can use a Gaussian filter to add a blurring effect to an image. The filter smoothens the transition between the zerores and the 1 by adding intermediate values. The sigma determines the slope of the filter, and thereby the smoothness of the transition.

<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/f/f6/Gaussian_Filter.svg/700px-Gaussian_Filter.svg.png" width="300" align="left">

In [ ]:
## Here we apply to filter to the matrix X and save it as a variable Y. 
## Change the value of sigma and describe what happens

y = ndi.filters.gaussian_filter(x, sigma=5)
plt.imshow(y, interpolation='none', cmap='gray')

In [ ]:
# The code in this cell applies a Guassian Filter to an image of the band Blur.
# QUESTION: Which features are highlighted by a Guassian Filter? 
blur = io.imread(join("images", "blur.jpg"), as_gray=True)
blurred_blur = ndi.filters.gaussian_filter(blur, sigma=5)

plt.figure(figsize=(20,20))
plt.subplot(1,2,1)
plt.title('Blur')
plt.imshow(blur, cmap='gray')
plt.axis('off')
plt.subplot(1,2,2)
plt.title('Blurred Blur')
plt.imshow(blurred_blur, cmap='gray')
plt.axis('off')

### Sobel Operator
The sobel operator can be used to highlight edges in an image. 
For more on the operator: https://en.wikipedia.org/wiki/Sobel_operator



In [ ]:
# Here we initialize two kernels. 
# One for horizontal edge detection (h_kernel) and one for vertical edge detection (v_kernel)

h_kernel = np.array([[ 1,  2,  1],
                     [ 0,  0,  0],
                     [-1, -2, -1]])

v_kernel = np.array([[ -1,  0,  1],
                     [ -2,  0,  2],
                     [-1, 0, 1]])

In [ ]:
# Plot the kernels
plt.figure(figsize=(20,20))
plt.subplot(1,2,1)
plt.title('Horizontal Edge Detection (h_kernel)')
plt.imshow(h_kernel, cmap='gray')
plt.axis('off')
plt.subplot(1,2,2)
plt.title('Vertical Edge Detection (v_kernel)')
plt.imshow(v_kernel, cmap='gray')
plt.axis('off')

In [ ]:
# Here we load the image of the train as a grayscale image

grayscale_train = io.imread(join("images", "color_train.jpg"), as_gray=True)
plt.imshow(grayscale_train, cmap='gray')

In [ ]:
# Apply the Sobel operator
Gx = convolve2d(grayscale_train, h_kernel)
Gy = convolve2d(grayscale_train, v_kernel)
G = np.sqrt(Gx**2 + Gy**2)
plt.imshow(G, cmap='gray')

## Building your own Convolutional Neural Network

Convolution Neural Networks are able to detect patterns in pixel values. They do this by learning (Artificial Intelligence) significant similarities and differences in a collection of images. As we will see below, convolutions play an important role in the architecture, the specific sequence of different layers, of a convolutional neural network. 

We will start by building our own classifier. On of the first convolutional neural networks was used to recognize handwritten digitis. The network was trained on data from the MNIST dataset, which contains 60,000 training and 10,000 testing images. We use Keras, a open-source Python framework for building neural networks to start building our own convolutional neural network.

If you feel you do not understand how convolutional neural networks work consider watching <a href="https://www.youtube.com/watch?v=CsN-qwpF8Bc" /> this short (somewhat embrassing) movie </a> I (Thomas) made for the Universiteit van Nederland (Dutch). 

In [ ]:
# Loads the MNIST dataset
from keras.datasets import mnist

### Preparing the data

In [ ]:
# Loads the training and testing images.
# QUESTION What is the difference between X_train and Y_train? 

(X_train, y_train), (X_test, y_test) = mnist.load_data('/tmp/mnist.npz')

In [ ]:
# Shows the shape (structure) of the X_train and X_Test set.
# QUESTION What do these values indicate?

print("Shape of X_Train: ", X_train.shape)
print("Shape of X_Test: ", X_test.shape)


In [ ]:
# Shows the first entry in the X_train dataset
# QUESTION: Which number do you have to fill in if you would want to print the 10th entry? Why?

X_train[0] 

In [ ]:
# Shows the images of the first entry in the X_train set. 
plt.imshow(X_train[0], cmap='gray')

In [ ]:
# Reshapes the images of the X_train set to 28X28 pixels. 
# QUESTION: Think of kernel going over the image (especially its edges). Why do we reshape the images into squares?

X_train = X_train.reshape(-1, 28*28)
X_test = X_test.reshape(-1, 28*28)

In [ ]:
# Shows the new shape of the X_train set.
X_train.shape

In [ ]:
# Transform the numbers of the dataset from integers to floats . 
# WHY DO WE DO THIS?
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255.0
X_test /= 255.0

In [ ]:
X_train[0]

In [ ]:
# Imports a function for creating a dummy encoding. 
# http://www.statisticssolutions.com/dummy-coding-the-how-and-why/
from keras.utils.np_utils import to_categorical

In [ ]:
# Here we see the annotated label (y_train[0]) for the first image (X_train[0]), which resembled a five.
y_train[0]

In [ ]:
# Here we turn the y variable into dummy variables. 
# go to 10 binary columns
y_train_cat = to_categorical(y_train)
y_test_cat = to_categorical(y_test)

In [ ]:
print("Shape of Y_train: ", y_train_cat.shape)
print("Shape of Y_test: ", y_test_cat.shape)

In [ ]:
# This shows the dummy encoding for the first entry. 
# QUESTION What number does this signify? 
y_train_cat[0]

In [ ]:
# QUESTION What number does this array signify?
y_train_cat[30]

### Building the Network

In [ ]:
# Imports several functions from the Keras library.
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import MaxPool2D, AvgPool2D, Conv2D
from keras.layers import Flatten, Activation
import keras.backend as K

In [ ]:
X_train = X_train.reshape(-1, 28, 28, 1)
X_test = X_test.reshape(-1, 28, 28, 1)

In the cell below, we initialize the architecture of our network. It consists of different kinds of layers. 
We already know the convolutional layer: our model has 32 filters in one layer.

### One Convolution layer
<img src="https://i.stack.imgur.com/xdXTn.gif">

### One Max Pooling layer
 (Max)Pooling layers reduce the complexity of the information of the image. 
<img src="https://computersciencewiki.org/images/8/8a/MaxpoolSample2.png">

In [ ]:
K.clear_session()

model = Sequential()

model.add(Conv2D(32, (3, 3), input_shape=(28, 28, 1))) #convolution layer 32 filters (3 x 3 pixels), output 26x26
model.add(MaxPool2D(pool_size=(2, 2))) # pooling layer to reduce size output is 13x13 
model.add(Activation('relu')) 

model.add(Flatten()) #flatten tensor into long array

model.add(Dense(128, activation='relu')) 

model.add(Dense(10, activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [ ]:
# This cell describes the architecture of our network (model)
model.summary()

In [ ]:
# Here we train the initialized model on the trained data of MNIST (the hand-written digits)

h = model.fit(X_train, y_train_cat, batch_size=128,
          epochs=3, verbose=1, validation_split=0.3)

In [ ]:
# Here we evaluate the model against the test data

plt.plot(h.history['acc'])
plt.plot(h.history['val_acc'])
plt.legend(['Training', 'Validation'])
plt.title('Accuracy')
plt.xlabel('Epochs')

test_accuracy = model.evaluate(X_test, y_test_cat)[1]
test_accuracy

In [ ]:
# Here we are going to test whether our network works. 
# The label of the 151st entry is a four. 
y_train[150]

In [ ]:
# Let's ask our model to predict the value of the 151st entry. 

model.predict_classes(X_train[150].reshape(-1, 28, 28, 1))[0]

# Try some other entries yourselves! :)

## Image classification

Convolutional neural networks are often used for image and/or object classification. The task of networks is to flag an image in a set if it contains a specific object, scene, colour scheme etc etc (image classification) or detect and 'box' a specific number of predetermined object categories (see image below)
<br><br><br><img src="https://www.pyimagesearch.com/wp-content/uploads/2017/09/example03_result.jpg" /><p style="text-align:center;"> A person riding a horse and two potted plants are successfully identified despite a lot of objects in the image via deep learning-based object detection
</p>
<p style="text-align:center;">
(borrowed from [Object detection with deep learning](https://www.pyimagesearch.com/2017/09/11/object-detection-with-deep-learning-and-opencv/))</p>


In [ ]:
# Loads functions from Keras library
from keras.applications.vgg19 import VGG19
from keras.preprocessing import image
from keras.applications.vgg19 import preprocess_input, decode_predictions
from keras.models import Model

## Using VGG19 for image classification

The VGG19 model was trained to identify 1000 classes of objects within an image. It was built as part of the ImageNet challenge, one of the most influential computer vision competitions that has been running since 2010.


<img src="https://lihan.me/assets/images/vgg-hero-cover.jpg"/>

<p style="text-align:center;">
__VGG19 Architecture__
</p>
<p style="text-align:center;">
(borrowed from [vgg19-caltech101-classification](https://lihan.me/2018/01/vgg19-caltech101-classification/))
</p>

VGG models described in the [original paper](https://arxiv.org/pdf/1409.1556.pdf) are trained with images whose size is 224x224x3. This can not be changed, so the input image for the transfer learning task should have the same image size.

In [ ]:
# Here we load the VGG19 model trained on the imagenet data. 
vgg19_full = VGG19(weights='imagenet')
vgg19_full.summary()

In [ ]:
# Loads an image of a 'bloemkool' from the folder 'images'
# QUESTION: Can you load your own image?
img_path = join("images", "bloemkool.jpg")
img = image.load_img(img_path, target_size=(224, 224)) #image needs to be 224 x 224
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)
plt.imshow(img)

In [ ]:
# This cell uses VGG19 to predict what the image shows.
# QUESTION: Can you predict the category of a random image from the internet using this notebook?
prediction = vgg19_full.predict(x)
print(prediction.shape)
for pred in decode_predictions(prediction)[0]:
        print(pred)


## Train your own classifier

We are going to train a model that can identify "bakfietsen".

<img src="https://images4.persgroep.net/rcs/BA5IZl03LajJ8m9wiriELowxFps/diocontent/68923602/_crop/0/135/1580/893/_fitwidth/694/?appId=21791a8992982cd8da851550a453bd7f&quality=0.9">

Here we load a different model, namely inception V3, which was developed by Google. Inception is faster than VGG19 when training your own classifier.

In [ ]:
## TO DO: ADD MORE IMAGES (SMALLER) TO IMPROVE CLASSIFIER

In [ ]:
from keras.applications import inception_v3

In [ ]:
# Instantiate the architecture, with pre-trained parameters from ImageNet
# but without the final (top) layer of object classification

inception_model = inception_v3.InceptionV3(include_top=False, pooling='avg', weights='imagenet')

In [ ]:
def extract_features(img):
    x = np.expand_dims(img.astype(np.float32), axis=0)  # Make a batch of 1 image by expanding the first dimension
    x = inception_v3.preprocess_input(x)  # Preprocess the input image ([0,255] range to [-1,+1])
    return inception_model.predict(x)[0]  # Make the batch goes through the network and convert the batch back to single image

In [ ]:
from glob import glob
from tqdm import tqdm

# Here we load our training, test, and random images into a list variable. 


# Gather the image files in each directory
filenames_bakfiets = glob('data/bakfiets/*.jpg')
print("Number of training positive files : {}".format(len(filenames_bakfiets)))
filenames_random = glob('data/random/*.jpg')
print("Number of training negative files : {}".format(len(filenames_random)))
filenames_test = glob('data/test/*.jpg')
print("Number of test files : {}".format(len(filenames_test)))

In [ ]:
features = []
labels = []
# Extracting features for the positive images
for filename in tqdm(filenames_bakfiets, desc='Computing features for bakfiets images'):
    img = io.imread(filename)
    # Add the CNN features to the list
    features.append(extract_features(img))
    # Add the positive label to the list
    labels.append(1.)
# Extracting features for the negative images
for filename in tqdm(filenames_random, desc='Computing features for random images'):
    img = io.imread(filename)
    # Add the CNN features to the list
    features.append(extract_features(img))
    # Add the negative label to the list
    labels.append(0.)
features = np.stack(features)
labels = np.array(labels)

In [ ]:
print("Number of images: ", features.shape[0])
print("Number of features per image: ", features.shape[1])
print("\n")
print("Features of image 1: ", features[0])

In [ ]:
# This is the list of labels, 1 signifies a bakfiets, 0 signifies something different. 
print(labels)

In [ ]:
# We are now going to train a classifier that can predict (based on the features) 
# whether an image is a bakfiets (1) or something different (0)

from sklearn.svm import SVC

# Instantiate a classifier that will predict a probability
classifier = SVC(probability=True, kernel='linear')
# Train the binary classifier from the extracted features and telling it which images are the good ones with the labels
classifier.fit(features, labels)

In [ ]:
# For each image in the directory with test data
for filename in tqdm(filenames_test, desc="Predicting"):
    # Load the image
    img = io.imread(filename)
    # Extract the features
    feat = extract_features(img)
    # Get the probabilities for the computed features
    probs = classifier.predict_proba(np.expand_dims(feat, axis=0))[0]
    prob_bakfiets = probs[1]
    # Display the results
    plt.figure()
    plt.imshow(img)
    plt.title("Probability : {:.01f}%".format(100*prob_bakfiets))

## Face detection

An important task of computer vision is face detecting. In the past, face detection was achieved by recognizing the relative distance between 'facial markers'. In recent years, machine learning has also greatly improved the task of face detection.<br><br><br><img src="https://i.dailymail.co.uk/i/pix/2015/02/18/25CCD2F400000578-2958597-image-a-27_1424270103152.jpg" /><p style="text-align:center;">
__Face Detection__
</p>
<p style="text-align:center;">
(Stolen from [A British tabloid](https://pleasedonotreportus.com/))
</p>

In [ ]:
# Imports the face_recognition library
import face_recognition as fr

In [ ]:
# Load the image face.jpg from the folder 'faces'.
img_path = join('images', 'faces', 'face.jpg')
img = fr.load_image_file(img_path)
plt.imshow(img)

In [ ]:
# Here we extract the locations of the faces and store them in the variable face_locations
face_locations = fr.face_locations(img, number_of_times_to_upsample=0, model='cnn')

In [ ]:
face_locations

In [ ]:
#Here we use the location information stored in face_locations to draw rectangles on top of the image.

fig,ax = plt.subplots(1,1)
plt.imshow(img)
n, m, d = img.shape
for face in face_locations:
    rect = plt.Rectangle((face[3], face[0]), face[2] - face[0], face[1] - face[3],
                         edgecolor='orange', linewidth=2, facecolor='none')
    ax.add_patch(rect)

## Thank you! 

If you have any questions please email us or visit our websites.

<a href="https://melvinwevers.github.io/" />Melvin Wevers </a> - melvin.wevers@dh.huc.knaw.nl <br>
<a href="http://illustratednewspictures.tumblr.com/" /> Thomas Smits </a> - thomassmits@hotmail.com